In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f'''
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.a_winare_dc_open_order_store AS
    WITH SUPRA AS (
        SELECT DISTINCT dc_id FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_dc_golive 
        WHERE 
            supra = 'YES'
            AND dc_id IN ('1291', '1312', '1352', '1354')
    ),
    OPEN_ORDER_SAP AS (
        SELECT 
            a.store_id,
            a.product_id,
            IFNULL(SUM(a.po_qty * c.numerator / c.denomintR), 0) open_order
        FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_item a
            LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_unit_conv c ON a.product_id = c.material AND a.po_unit = c.mat_unit
        WHERE a.creation_date BETWEEN DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 21) AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
            AND (a.compl_del <> 'X' OR a.compl_del IS NULL)
            AND (a.po_gr_date IS NULL)
            AND (a.gr_date IS NULL OR (a.gr_date IS NOT NULL AND a.gr_qty = 0))
            AND (a.webportal_status NOT IN ('Đã hủy', 'Đã giao hàng', 'Quá hạn') OR a.webportal_status IS NULL)
            AND (a.dc_id NOT IN (SELECT * FROM SUPRA) OR a.dc_id IS NULL)
        GROUP BY 1,2
    ), 
    OPEN_ORDER_SAP_FULL AS (
        SELECT 
            a.store_id,
            a.product_id,
            IFNULL(SUM(a.po_qty * c.numerator / c.denomintR), 0) open_order,
            CONCAT_WS(', ', COLLECT_LIST(CONCAT(a.po_doc, '-', IFNULL(a.dc_id, LTRIM('0',a.vendor)), '-', CAST(IFNULL(a.po_qty * c.numerator / c.denomintR, 0) AS STRING)))) po_doc
        FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_item a
            LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_unit_conv c ON a.product_id = c.material AND a.po_unit = c.mat_unit
        WHERE a.creation_date BETWEEN DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 21) AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
            AND (a.compl_del <> 'X' OR a.compl_del IS NULL)
            AND (a.po_gr_date IS NULL)
            AND (a.gr_date IS NULL OR (a.gr_date IS NOT NULL AND a.gr_qty = 0))
            AND (a.webportal_status NOT IN ('Đã hủy', 'Đã giao hàng', 'Quá hạn') OR a.webportal_status IS NULL)
        GROUP BY 1,2
    ), 
    OPEN_ORDER_SAP_EXTERNAL AS (
        SELECT 
            a.store_id,
            a.product_id,
            IFNULL(SUM(a.po_qty * c.numerator / c.denomintR), 0) open_order,
            CONCAT_WS(', ', COLLECT_LIST(CONCAT(a.po_doc, '-', IFNULL(a.dc_id, LTRIM('0',a.vendor)), '-', CAST(IFNULL(a.po_qty * c.numerator / c.denomintR, 0) AS STRING)))) po_doc
        FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_item a
            LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_unit_conv c ON a.product_id = c.material AND a.po_unit = c.mat_unit
            LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store d ON a.dc_id = d.store_id
        WHERE a.creation_date BETWEEN DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS),21) AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
            AND (a.compl_del <> 'X' OR a.compl_del IS NULL)
            AND (a.po_gr_date IS NULL)
            AND (a.gr_date IS NULL OR (a.gr_date IS NOT NULL AND a.gr_qty = 0))
            AND (a.webportal_status NOT IN ('Đã hủy', 'Đã giao hàng', 'Quá hạn') OR a.webportal_status IS NULL)
            AND (d.business_unit = '1100' OR a.dc_id IS NULL)
        GROUP BY 1,2
    ), 
    OPEN_ORDER_FC AS (
        SELECT      
            a.store_id,
            a.product_id,
            SUM(IFNULL(base_qty,0)) open_order
        FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.f_po_pending_fc a
        WHERE creation_date BETWEEN DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 21) AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
        GROUP BY 1,2
    ), 
    OPEN_ORDER_SFT AS (
        SELECT      
            store_id,
            product_id,
            SUM(open_order) open_order
        FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_store_open_order_sft
        WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
            AND dc_id IN (SELECT * FROM SUPRA)
        GROUP BY 1,2
    ), 
    STORE_SKU_LIST AS (
        SELECT store_id, product_id
        FROM OPEN_ORDER_SAP
        UNION DISTINCT
        SELECT store_id, product_id
        FROM OPEN_ORDER_FC
        UNION DISTINCT
        SELECT store_id, product_id
        FROM OPEN_ORDER_SFT 
        UNION DISTINCT
        SELECT store_id, product_id
        FROM OPEN_ORDER_SAP_FULL           
    )
    SELECT     
        DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1 calday,
        a.store_id,
        a.product_id,
        21 sto_pending_day,
        IFNULL(a1.open_order, 0) open_order_sap,
        IFNULL(a4.open_order, 0) open_order_sap_full,
        IFNULL(a5.open_order, 0) open_order_sap_external,
        a4.po_doc,
        IFNULL(a2.open_order, 0) open_order_fc,
        IFNULL(a3.open_order, 0) open_order_sft,
        IFNULL(a2.open_order, 0) + IFNULL(a4.open_order, 0) open_order
    FROM STORE_SKU_LIST a
    LEFT JOIN OPEN_ORDER_SAP a1 USING (store_id, product_id)
    LEFT JOIN OPEN_ORDER_FC a2 USING (store_id, product_id)
    LEFT JOIN OPEN_ORDER_SFT a3 USING (store_id, product_id)
    LEFT JOIN OPEN_ORDER_SAP_FULL a4 USING (store_id, product_id)
    LEFT JOIN OPEN_ORDER_SAP_EXTERNAL a5 USING (store_id, product_id)
    ''')